# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import joblib
import pandas as pd
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="lol_match_predictor")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-146414
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-146414


In [2]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
data = pd.read_csv("https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20210126.csv")

# Show raw data before it is processed.
data.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,118.0,165.0,166.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,98.0,-399.0,150.0,-7.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,140.0,-409.0,-1837.0,-11.0,0.0,0.0,1.0,0.0,1.0,0.0
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,135.0,51.0,-401.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,28.0,-233.0,257.0,-8.0,0.0,0.0,0.0,0.0,1.0,0.0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [14]:
#TODO: Create the different params that you will be using during training
ps = BayesianParameterSampling( {
        "--C": uniform(0.1,10),
        "--max_iter": choice(20,40,60,80,100,150,200,300,400,600,800)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
est = SKLearn(".", compute_target=compute_target, entry_script="train.py")

hyperdrive_config = HyperDriveConfig(ps,
                                     estimator=est,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)






In [30]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [31]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [50]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '6.761318577992516', '--max_iter', '200']


In [51]:
#TODO: Save the best model
#model = best_run.register_model(model_name="predictor", model_path="outputs/model.pkl")
#print(model)
best_run.download_file("outputs/model.pkl", "model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [52]:
model = Model.register(model_path = "model.pkl",
                       model_name = "matchpredictor",
                       description = "League of Legends match predictor trained using Azure ML.",
                       workspace = ws)

Registering model matchpredictor


In [38]:
print(os.getenv('AZUREML_MODEL_DIR'))

None


In [53]:
# Get environment
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Create inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Set deployment configuration as an Azure Container Instance
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)

# Service deployment
service = Model.deploy(
    workspace=ws,
    name="lolpredictor",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-03 13:53:43+00:00 Creating Container Registry if not exists.
2021-06-03 13:53:43+00:00 Registering the environment.
2021-06-03 13:53:45+00:00 Use the existing image.
2021-06-03 13:53:45+00:00 Generating deployment configuration.
2021-06-03 13:53:46+00:00 Submitting deployment to compute..
2021-06-03 13:53:50+00:00 Checking the status of deployment lolpredictor..
2021-06-03 13:57:43+00:00 Checking the status of inference endpoint lolpredictor.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [54]:
scoring_uri = service.scoring_uri

print("Service State: {}".format(service.state))
print("Scoring URI: {}".format(service.scoring_uri))
print("Swagger URI: {}".format(service.swagger_uri))


Service State: Healthy
Scoring URI: http://2315b435-da73-4051-ab38-2fd24f2dedf8.southcentralus.azurecontainer.io/score
Swagger URI: http://2315b435-da73-4051-ab38-2fd24f2dedf8.southcentralus.azurecontainer.io/swagger.json


In [55]:
import requests
import json
import numpy as np

scoring_uri = service.scoring_uri

data = {'data': 
            [ {
                "golddiffat10": -500,
                "xpdiffat10": -300,
                "golddiffat15": 2000,
                "xpdiffat15": 1600
                },
            ]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}

response = requests.post(scoring_uri, input_data, headers=headers)
print(response.text)

"{\"result\": [1]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [56]:

print(service.get_logs())

2021-06-03T13:57:36,292011000+00:00 - iot-server/run 
2021-06-03T13:57:36,298679700+00:00 - rsyslog/run 
2021-06-03T13:57:36,326444900+00:00 - gunicorn/run 
2021-06-03T13:57:36,375804400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_cab002048cdaff7e785b2a3884f6881a/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_cab002048cdaff7e785b2a3884f6881a/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_cab002048cdaff7e785b2a3884f6881a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_cab002048cdaff7e785b2a3884f6881a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_cab002048cdaff7e785b2a3884f6881a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [57]:
service.delete()